In [ ]:
import os
import numpy as np
import shutil
import random
import cv2 as cv2

In [ ]:

#Function to split folders of classes to Train , Validation and Test
# SEED is used for reproducible splitting
# IF root dir or/and destination dir are not defined then current directory of project will be used
# 
#Format is like :
#INPUT:
# root_dir/
#   class1/
#       img1.jpg
#       img2.jpg
#       ...
#   class2/
#       img1.jpg
#       img2.jpg
#       ...
#   class3/
#       img1.jpg
#       img2.jpg
#       ...
#   ...
#
#OUTPUT:
# destination_dir/
#    train/
#        class1/
#            img1.jpg
#            ...
#        class2/
#            img1.jpg
#            ...
#    val/
#        class1/
#            img2.jpg
#            ...
#        class2/
#            imgb.jpg
#            ...
#    test/
#        class1/
#            img3.jpg
#            ...
#        class2/
#           imgc.jpg
#            ...

def folder_splitting_dataset(classes_dir,val_ratio,test_ratio,SEED,root_dir=None,destination_dir=None):
    if root_dir==None:
        root_dir=os.getcwd()
    if destination_dir==None:
        destination_dir=root_dir
    np.random.seed(SEED)


    for cls in classes_dir:
        os.makedirs(root_dir +'/train' + '/' + cls)
        os.makedirs(root_dir +'/val' + '/' + cls)
        os.makedirs(root_dir +'/test' + '/' + cls)


        # Creating partitions of the data after shuffeling
        src = root_dir + '/' + cls  # Folder to copy images from

        allFileNames = os.listdir(src)
        allFileNames.sort()
        np.random.shuffle(allFileNames)
        train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                                  [int(len(allFileNames)* (1 - (val_ratio + test_ratio))), 
                                                                   int(len(allFileNames)* (1 - test_ratio))])


        train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
        val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
        test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]
        print('Class Name: ',cls)
        print('Total images: ', len(allFileNames))
        print('Training: ', len(train_FileNames))
        print('Validation: ', len(val_FileNames))
        print('Testing: ', len(test_FileNames))

        # Copy-pasting images
        print("Copying Images..")
        for name in train_FileNames:
            shutil.copy(name, destination_dir +'/train'+ '/' + cls)

        for name in val_FileNames:
            shutil.copy(name, destination_dir +'/val'+ '/' + cls)

        for name in test_FileNames:
            shutil.copy(name, destination_dir +'/test'+ '/' + cls)
        print("Class Completed\n")

In [ ]:
classes_dir=['Disappointed','interested','neutral']
folder_splitting_dataset(classes_dir=classes_dir,val_ratio=0.2,test_ratio=0.1,SEED=20)

In [ ]:
root_dir=os.getcwd
classes_dir=['Disappointed','interested','neutral']
split_sets=['train,']

In [ ]:
def load_dataset(img_folder,grayScaled,normalized,IMG_HEIGHT,IMG_WIDTH):
   
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
       
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread( image_path )
            if grayScaled:
                image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
            image=np.array(image)
            image = image.astype('float32')
            if normalized:
                image /= 255 
                
            img_data_array.append(image)
            class_name.append(dir1)
    return img_data_array, class_name

In [ ]:
img_data, class_name =load_dataset("C:\\Users\\Stelios\\Desktop\\FER_Custom_Dataset\\FER_Split\\val",True,False,64,64)
valX=np.array(img_data)
dictY={k: v for v, k in enumerate(np.unique(class_name))}
valY=  [dictY[class_name[i]] for i in range(len(class_name))]
np.save('valX.npy', valX)
np.save("valY.npy",valY)

img_data, class_name =load_dataset("C:\\Users\\Stelios\\Desktop\\FER_Custom_Dataset\\FER_Split\\train",True,False,64,64)

trainX=np.array(img_data)
dictY={k: v for v, k in enumerate(np.unique(class_name))}
trainY=  [dictY[class_name[i]] for i in range(len(class_name))]
np.save('trainX.npy', trainX)
np.save("trainY.npy",trainY)

img_data, class_name =load_dataset("C:\\Users\\Stelios\\Desktop\\FER_Custom_Dataset\\FER_Split\\test",True,False,64,64)
testX=np.array(img_data)
dictY={k: v for v, k in enumerate(np.unique(class_name))}
testY=  [dictY[class_name[i]] for i in range(len(class_name))]
np.save('testX.npy', testX)
np.save("testY.npy",testY)

In [ ]:
test=np.array(list(map(int,testY)), np.float32)
test.shape